In [ ]:
!pip install flair==0.4.3

In [ ]:
!wget -c http://gemeos.org/henrique/health_fasttext_300v1.tar.gz -O health_fasttext_300v1.tar.gz #0.86
!tar -zxvf health_fasttext_300v1.tar.gz

--2020-02-06 13:16:30--  http://gemeos.org/henrique/health_fasttext_300v1.tar.gz
Resolving gemeos.org (gemeos.org)... 187.45.195.133
Connecting to gemeos.org (gemeos.org)|187.45.195.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2255702828 (2.1G) [application/x-gzip]
Saving to: ‘health_fasttext_300v1.tar.gz’

health_fasttext_300 100%[===================>]   2.10G  15.5MB/s    in 2m 31s  

2020-02-06 13:19:01 (14.3 MB/s) - ‘health_fasttext_300v1.tar.gz’ saved [2255702828/2255702828]

health_fasttext_300.model
health_fasttext_300.model.trainables.syn1neg.npy
health_fasttext_300.model.trainables.vectors_ngrams_lockf.npy
health_fasttext_300.model.trainables.vectors_vocab_lockf.npy
health_fasttext_300.model.wv.vectors_ngrams.npy
health_fasttext_300.model.wv.vectors.npy
health_fasttext_300.model.wv.vectors_vocab.npy


## Token Classification

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.datasets import CSVClassificationCorpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings
from flair.embeddings import DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.optim import SGDW
from typing import List

pathCorpus : str = '/content/Fall-Recognition/Data/'
train_file : str = 'CoNLL-train.txt'
test_file : str = 'CoNLL-test.txt'
dev_file : str = 'CoNLL-dev.txt'
pathCheckpoint: str = '/content/Fall-Recognition/Experiments/NER-Fall/resources/taggers/example-ner'
pathWordEmbeddings : str = 'health_fasttext_300.model'
pathFlairEmbeddingsForward : str = None
pathFlairEmbeddingsBackward : str =  None

print('--------------------------START TRAINING (TOKEN)------------------------------')

columns = {0:'token', 1:'label'}

corpus: Corpus = ColumnCorpus(pathCorpus, columns,
	train_file = train_file,
	test_file = test_file,
	dev_file = dev_file)

print(" ")
print("Train len: ", len(corpus.train))
print("Test len: ", len(corpus.test))
print("Dev len: ", len(corpus.dev))

tag_type = 'label'
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

print(tag_dictionary.idx2item)
print("len: ", len(tag_dictionary.idx2item))

health_embedding = WordEmbeddings(pathWordEmbeddings)

embedding_types: List[TokenEmbeddings] = [
	health_embedding,
]
  
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)
	
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
										embeddings=embeddings,
										tag_dictionary=tag_dictionary,
										tag_type=tag_type,
										use_crf=True)

trainer: ModelTrainer = ModelTrainer(tagger, corpus, optimizer=SGDW)

trainer.train(pathCheckpoint,
			  learning_rate=0.1,
			  mini_batch_size=32,
			  max_epochs=150,
			  checkpoint=True)

--------------------------START TRAINING (TOKEN)------------------------------
2020-02-05 14:26:33,584 Reading data from /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data
2020-02-05 14:26:33,588 Train: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/CoNLL-train.txt
2020-02-05 14:26:33,590 Dev: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/CoNLL-dev.txt
2020-02-05 14:26:33,591 Test: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/CoNLL-test.txt
 
Train len:  1122
Test len:  210
Dev len:  70
[b'<unk>', b'O', b'B-QUEDA', b'I-QUEDA', b'<START>', b'<STOP>']
len:  6


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2020-02-05 14:28:48,089 ----------------------------------------------------------------------------------------------------
2020-02-05 14:28:48,091 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('health_fasttext_300.model')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, bidirectional=True)
  (linear): Linear(in_features=512, out_features=6, bias=True)
)"
2020-02-05 14:28:48,092 ----------------------------------------------------------------------------------------------------
2020-02-05 14:28:48,093 Corpus: "Corpus: 1122 train + 70 dev + 210 test sentences"
2020-02-05 14:28:48,095 ----------------------------------------------------------------------------------------------------
2020-02-05 14:28:48,096 Parameters:
2020-02-05 14:28:48,098  - learning_rate: "0.1"
2020-02-05 14:28:48,100  - mini_batch_size:

{'test_score': 0.625,
 'dev_score_history': [0.4571,
  0.6122,
  0.6087,
  0.6977,
  0.7143,
  0.619,
  0.6364,
  0.6364,
  0.6512,
  0.6531,
  0.6122,
  0.6531,
  0.6531,
  0.6809,
  0.6531,
  0.6957,
  0.6667,
  0.6531,
  0.6531,
  0.6667,
  0.6667,
  0.6667,
  0.6531,
  0.6531,
  0.6531,
  0.6531,
  0.6531,
  0.6531,
  0.6667,
  0.6531,
  0.6531,
  0.6531,
  0.6667,
  0.6531,
  0.6531,
  0.6531,
  0.6531,
  0.6531,
  0.6531,
  0.6531,
  0.6531,
  0.6531,
  0.6531,
  0.6531,
  0.6531],
 'train_loss_history': [54.65205971399943,
  2.4639715221193104,
  2.0710758103264704,
  1.8496916426552668,
  1.7805431948767767,
  1.5284578932656183,
  1.4489326344596014,
  1.5892141924964056,
  1.4728737009896173,
  1.3702501588397555,
  1.3202798896365695,
  1.2392514414257474,
  1.3136735757191975,
  1.220424042807685,
  1.2426301108466253,
  1.1121456887986925,
  1.1250076293945312,
  1.0950231552124023,
  1.0758613877826266,
  1.1171122524473402,
  1.1624361673990886,
  1.0519966284434001,
  1

In [ ]:
from EvaluatesModels import EvaluatesTokenClassification

tkc_inner = EvaluatesTokenClassification(
		path_model_token='/content/Fall-Recognition/Experiments/NER-Fall/resources/taggers/example-ner/best-model.pt',
		path_valid_data='/content/Fall-Recognition/Data/Text-Test.csv')

tkc_inner.make_validation_token_classification()

2020-02-17 12:37:04,313 loading file /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Experiments/NER-Fall/resources/taggers/example-ner/best-model.pt
 
--------------------------START VALIDATION TOKEN------------------------------
 
 
 --- VALID --- 
              precision    recall  f1-score   support

     class 0       0.91      0.99      0.95       141
     class 1       0.96      0.80      0.87        69

    accuracy                           0.92       210
   macro avg       0.94      0.89      0.91       210
weighted avg       0.93      0.92      0.92       210

	F1 Score

Class 0	 0.945578231292517
Class 1	 0.8730158730158731


## CLinical note Classification

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.datasets import CSVClassificationCorpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings
from flair.embeddings import DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.optim import SGDW
from typing import List

pathCorpus : str = '/content/Fall-Recognition/Data/'
pathWordEmbeddings : str = 'health_fasttext_300.model'
train_file : str = 'Text-Train.csv'
test_file : str = 'Text-Test.csv'
dev_file : str = 'Text-Dev.csv'
pathCheckpoint: str = '/content/Fall-Recognition/Experiments/TxC-Fall/resources/taggers/example-txc'
pathFlairEmbeddingsForward : str = None
pathFlairEmbeddingsBackward : str =  None

print('--------------------------START TRAINING (TEXT)------------------------------')

column_name_map = {0: "text", 1: "label"}

corpus: Corpus = CSVClassificationCorpus(pathCorpus,
										column_name_map,
										train_file = train_file,
										test_file = test_file,
										dev_file = dev_file,
										skip_header = True,
										delimiter = '\t',
) 

#Verbose Corpus:
print(corpus)

label_dict = corpus.make_label_dictionary()

health_embedding = WordEmbeddings(pathWordEmbeddings)

word_embeddings = [health_embedding]

document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
																	 hidden_size=512,
																	 reproject_words=True,
																	 reproject_words_dimension=256)

classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

trainer = ModelTrainer(classifier, corpus, optimizer=SGDW)

trainer.train(pathCheckpoint,
			  learning_rate=0.1,
			  mini_batch_size=32,
			  anneal_factor=0.5,
			  patience=5,
			  max_epochs=150,
			  checkpoint=True)

--------------------------START TRAINING (TEXT)------------------------------
2020-02-06 13:20:23,542 Reading data from /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data
2020-02-06 13:20:23,543 Train: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/Text-Train.csv
2020-02-06 13:20:23,543 Dev: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/Text-Dev.csv
2020-02-06 13:20:23,544 Test: /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Data/Text-Test.csv
Corpus: 1122 train + 70 dev + 210 test sentences
2020-02-06 13:20:25,640 Computing label dictionary. Progress:


100%|██████████| 1122/1122 [00:02<00:00, 474.27it/s]

2020-02-06 13:20:28,091 [b'1', b'0']



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2020-02-06 13:20:40,984 ----------------------------------------------------------------------------------------------------
2020-02-06 13:20:40,987 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('health_fasttext_300.model')
    )
    (word_reprojection_map): Linear(in_features=300, out_features=256, bias=True)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=512, out_features=2, bias=True)
  (loss_function): CrossEntropyLoss()
)"
2020-02-06 13:20:40,988 ----------------------------------------------------------------------------------------------------
2020-02-06 13:20:40,990 Corpus: "Corpus: 1122 train + 70 dev + 210 test sentences"
2020-02-06 13:20:40,992 ----------------------------------------------------------------------------------------------------
2020-02-06 13:20:40,993 Parameters:
2020-02-06 13:20:40,994  - learning_

{'dev_loss_history': [tensor(0.5360, device='cuda:0'),
  tensor(0.5538, device='cuda:0'),
  tensor(0.5448, device='cuda:0'),
  tensor(0.5673, device='cuda:0'),
  tensor(0.7731, device='cuda:0'),
  tensor(0.5699, device='cuda:0'),
  tensor(0.7594, device='cuda:0'),
  tensor(0.5601, device='cuda:0'),
  tensor(0.5328, device='cuda:0'),
  tensor(0.5487, device='cuda:0'),
  tensor(0.5267, device='cuda:0'),
  tensor(0.5210, device='cuda:0'),
  tensor(0.5225, device='cuda:0'),
  tensor(0.5694, device='cuda:0'),
  tensor(0.5117, device='cuda:0'),
  tensor(0.6171, device='cuda:0'),
  tensor(0.5540, device='cuda:0'),
  tensor(0.5438, device='cuda:0'),
  tensor(0.5081, device='cuda:0'),
  tensor(0.5090, device='cuda:0'),
  tensor(0.5473, device='cuda:0'),
  tensor(0.5018, device='cuda:0'),
  tensor(0.5263, device='cuda:0'),
  tensor(0.4995, device='cuda:0'),
  tensor(0.4803, device='cuda:0'),
  tensor(0.4742, device='cuda:0'),
  tensor(0.4885, device='cuda:0'),
  tensor(0.4727, device='cuda:0'),


In [ ]:
from EvaluatesModels import EvaluatesTextClassification

txc_inner = EvaluatesTextClassification(
		path_model_text='/content/Fall-Recognition/Experiments/TxC-Fall/resources/taggers/example-txc/best-model.pt',
		path_valid_data='/content/Fall-Recognition/Data/Text-Test.csv')

txc_inner.make_validation_text_classification()

2020-02-17 12:38:59,930 loading file /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Experiments/TxC-Fall/resources/taggers/example-txc/best-model.pt
 
--------------------------START VALIDATION TEXT------------------------------
 
 
 --- VALID --- 
              precision    recall  f1-score   support

     class 0       0.79      0.78      0.79       141
     class 1       0.56      0.58      0.57        69

    accuracy                           0.71       210
   macro avg       0.68      0.68      0.68       210
weighted avg       0.72      0.71      0.72       210

	F1 Score

Class 0	 0.7857142857142857
Class 1	 0.5714285714285715


## Validation Data

In [ ]:
!python3 EvaluatesModels.py

2020-02-07 16:30:25,443 loading file /content/drive/My Drive/Colab Notebooks/Fall-Recognition/Experiments/NER-Fall/resources/taggers/example-ner/best-model.pt
 
--------------------------START VALIDATION TOKEN------------------------------
 
 
 --- VALID --- 
              precision    recall  f1-score   support

     class 0       0.92      0.97      0.95      1913
     class 1       0.85      0.68      0.75       477

    accuracy                           0.91      2390
   macro avg       0.89      0.82      0.85      2390
weighted avg       0.91      0.91      0.91      2390

	F1 Score

Class 0	 0.9459734964322121
Class 1	 0.7523364485981309
